# 작업형 1유형

In [ ]:
'''
1. 다음은 California Housing 데이터 세트이다. 데이터 중 결측치가 있는 경우 해당 데이터의 행을 모두 제거하고, 첫번째 행부터 순서대로 70%까지의 데이터를 훈련 데이터로 추출한 데이터 세트를 구성한다. 변수 중 ‘housing_median_age’의 Q1(제1사분위수)값을 정수로 계산하시오.
'''

In [19]:
import pandas as pd
data = pd.read_csv('301_housing.csv')
# 결측치 제거
data = data[~data['total_bedrooms'].isnull()]
# 70%까지 추출
data2 = data.iloc[:int(0.7 * len(data)),:]
# 제1사분위수 추출
print(int(data2['housing_median_age'].quantile(0.25)))

19


In [ ]:
'''
2. 다음은 국가별 연도별 인구 10만명당 결핵 유병률 데이터세트이다. 2000년도의 국가별 결핵 유병률 데이터 세트에서 2000년도의 평균값보다 더 큰 유병률 값을 가진 국가의 수를 계산하시오.
'''

In [116]:
import pandas as pd 
data = pd.read_csv('302_worlddata.csv')
data2 = data[data['year']== 2000].drop(columns='year').T
#data2.columns = ['2000']
#print(data2[1] > data2[1])
print(len(data2[data2.loc[:,1] > data2.loc[:,1].mean()]))
#answer = (data2 > data2.mean()).sum()
# print(answer)
#print(answer[0])

76


In [ ]:
'''
3. 다음은 Titanic 데이터 세트이다. 주어진 데이터 세트의 컬럼 중 빈 값 또는 결측치를 확인하여, 결측치의 비율이 가장 높은 변수명을 출력하시오.
'''

In [71]:
import pandas as pd
data = pd.read_csv('303_titanic.csv')
print(data.isnull().sum().sort_values(ascending=False).index[0])

Age


# 작업형 2유형

In [ ]:
'''
다음은 Travel Insuarance 데이터 세트이다. 주어진 훈련 데이터 세트를 이용하여 고객별 여행보험 가입 여부 예측모형을 만들고, 가장 높은 Accuracy값을 가지는 최종모델을 도출하시오. 해당 모델을 활용하여 보험가입여부 예측값을 계산하고 결과값은 csv파일로 제출하시오.

* 결과 제출 양식 : ID, TravelInsurance
'''

In [275]:
import pandas as pd
x_train = pd.read_csv('304_x_train.csv')
y_train = pd.read_csv('304_y_train.csv')
x_test = pd.read_csv('304_x_test.csv')

# print(x_train.shape, y_train.shape, x_test.shape)
data = pd.concat([x_train, y_train.drop(columns=['ID'])], axis=1)


# 데이터 확인하기

# 라벨인코딩
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train['Employment Type'] = encoder.fit_transform(x_train['Employment Type'])
x_test['Employment Type'] = encoder.transform(x_test['Employment Type'])

encoder = LabelEncoder()
x_train['GraduateOrNot'] = encoder.fit_transform(x_train['GraduateOrNot'])
x_test['GraduateOrNot'] = encoder.transform(x_test['GraduateOrNot'])

encoder = LabelEncoder()
x_train['FrequentFlyer'] = encoder.fit_transform(x_train['FrequentFlyer'])
x_test['FrequentFlyer'] = encoder.transform(x_test['FrequentFlyer'])

encoder = LabelEncoder()
x_train['EverTravelledAbroad'] = encoder.fit_transform(x_train['EverTravelledAbroad'])
x_test['EverTravelledAbroad'] = encoder.transform(x_test['EverTravelledAbroad'])

# 전처리 (이상값, 결측치)

# 스케일링
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

x_train['Age'] = scaler.fit_transform(x_train[['Age']])
x_test['Age'] = scaler.transform(x_test[['Age']])

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

x_train['AnnualIncome'] = scaler.fit_transform(x_train[['AnnualIncome']])
x_test['AnnualIncome'] = scaler.transform(x_test[['AnnualIncome']])

id_column = x_test['ID']
x_test.drop(columns='ID', inplace=True)
x_train.drop(columns='ID', inplace=True)
y_train.drop(columns='ID', inplace=True)

# split
from sklearn.model_selection import train_test_split
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(x_train, y_train,test_size=0.2)

# model 학습
import xgboost

from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=300, max_depth=3, learning_rate=0.03)
model.fit(X_TRAIN, Y_TRAIN)
Y_TEST_PREDICTED = model.predict(X_TEST)

# 평가
from sklearn.metrics import roc_auc_score

# 모델 적용
model = XGBClassifier(n_estimators=300, max_depth=3, learning_rate=0.03)
model.fit(x_train, y_train)
y_test_predicted = model.predict(x_test)
y_test_predicted = pd.DataFrame({"TravelInsurance" : y_test_predicted})
result = pd.concat([id_column, y_test_predicted], axis=1)

# 제출
result.to_csv('12345.csv', index=False)